# Import libraries

In [12]:
import pandas as pd
import numpy as np 
import datetime
import time
import sys
from scipy import stats
from statsmodels.stats import stattools

# Include py file

In [2]:
from RiskLabAI.data.structures.data_structures_lopez import *

# Import Tick data

In [3]:
dir = "https://raw.githubusercontent.com/risk-labratory/data/main/"
url = dir + "IVE_2020.csv"
dataframe = pd.read_csv(url, header = 0)
dataframe['dates'] = pd.to_datetime(dataframe['dates'])
dataframe.set_index('dates', inplace = True, drop = True)
dataframe.drop_duplicates(inplace = True)
dataframe = dataframe[(dataframe.index.hour >= 9) & (dataframe.index.hour<16)]
dataframe.head()

,price,bid,ask,size
dates,,,,
2020-01-02 09:30:00,130.68,130.59,130.60,20625
2020-01-02 09:30:01,130.50,130.50,130.77,200
2020-01-02 09:30:04,130.53,130.52,130.78,100
2020-01-02 09:30:04,130.55,130.52,130.78,100
2020-01-02 09:30:04,130.53,130.52,130.78,200


In [4]:
ohlcvTime = generate_time_bar(dataframe, frequency = "30Min")
ohlcvTime.head(10)

C:\Users\harian\AppData\Roaming\Python\Python311\site-packages\RiskLabAI\data\structures\data_structures_lopez.py:164: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: (x['price'] * x['size']).sum() / x['size'].sum()


,open,high,low,close,volume,value_of_trades,price_mean,tick_count,price_mean_log_return
dates,,,,,,,,,
2020-01-02 09:30:00,130.6800,130.7800,130.3700,130.4299,164048,130.585010,130.558158,262,NaN
2020-01-02 10:00:00,130.4000,130.4000,130.0218,130.0564,75227,130.141558,130.178844,132,-0.002910
2020-01-02 10:30:00,130.0800,130.2000,129.9713,130.2000,91006,130.079069,130.056870,124,-0.000937
2020-01-02 11:00:00,130.2050,130.2089,130.0800,130.1300,65182,130.141968,130.138222,64,0.000625
2020-01-02 11:30:00,130.0833,130.0833,129.9380,130.0300,84253,130.013381,130.016007,59,-0.000940
2020-01-02 12:00:00,130.0215,130.1179,129.9800,130.0732,59420,130.063977,130.075780,60,0.000460
2020-01-02 12:30:00,130.0945,130.1348,130.0420,130.0773,152349,130.084798,130.073998,125,-0.000014
2020-01-02 13:00:00,130.0700,130.0700,129.9566,129.9566,16861,129.995596,129.990660,45,-0.000641
2020-01-02 13:30:00,129.9799,130.0200,129.9400,129.9779,64199,129.969085,129.969793,81,-0.000161


In [5]:
timeBarLen = ohlcvTime.shape[0]
ohlcvTick = generate_tick_bar(dataframe, ticks_per_bar = 0, number_bars = timeBarLen)
ohlcvTick.head(10)

,open,high,low,close,volume,value_of_trades,price_mean,tick_count,price_mean_log_return
dates,,,,,,,,,
2020-01-02 09:30:00,130.680,130.7800,130.5000,130.7800,23625,130.672466,130.607500,16,NaN
2020-01-02 09:30:05,130.780,130.7800,130.5200,130.6650,8446,130.647502,130.676562,16,0.000529
2020-01-02 09:30:29,130.660,130.7049,130.6000,130.6200,1982,130.649491,130.647181,16,-0.000225
2020-01-02 09:32:12,130.620,130.6200,130.5600,130.5700,5200,130.574894,130.579031,16,-0.000522
2020-01-02 09:32:51,130.570,130.6100,130.5567,130.5900,5721,130.589296,130.586669,16,0.000058
2020-01-02 09:33:04,130.590,130.6300,130.5900,130.6242,52946,130.609491,130.611506,16,0.000190
2020-01-02 09:33:32,130.640,130.6700,130.5350,130.5500,5886,130.589667,130.604900,16,-0.000051
2020-01-02 09:35:05,130.550,130.5900,130.4800,130.5600,3614,130.536972,130.544238,16,-0.000465
2020-01-02 09:35:54,130.570,130.6200,130.5000,130.5000,5021,130.549199,130.551112,16,0.000053


In [7]:
ohlcvVolume = generate_volume_bar(dataframe, volume_per_bar = 0, number_bars = timeBarLen)
ohlcvVolume.head(10)

,open,high,low,close,volume,value_of_trades,price_mean,tick_count,price_mean_log_return
dates,,,,,,,,,
2020-01-02 09:30:00,130.6800,130.6800,130.5000,130.5300,21225,130.675571,130.558000,5,NaN
2020-01-02 09:30:04,130.5400,130.7800,130.5200,130.7800,5913,130.716073,130.664444,18,0.000815
2020-01-02 09:30:12,130.5200,130.7400,130.5200,130.5700,8315,130.587541,130.632637,32,-0.000243
2020-01-02 09:32:50,130.5700,130.5900,130.5567,130.5900,6921,130.575836,130.576668,19,-0.000429
2020-01-02 09:33:04,130.6100,130.6200,130.5900,130.6200,6400,130.599844,130.602667,15,0.000199
2020-01-02 09:33:22,130.6100,130.6700,130.5980,130.6700,50546,130.611426,130.637436,14,0.000266
2020-01-02 09:34:09,130.6295,130.6295,130.5350,130.5400,6286,130.566144,130.564275,16,-0.000560
2020-01-02 09:35:17,130.4978,130.6200,130.4500,130.4900,7975,130.531469,130.528919,32,-0.000271
2020-01-02 09:39:53,130.5100,130.5600,130.5100,130.5300,7089,130.530707,130.530645,31,0.000013


In [8]:
ohlcvDollar = generate_dollar_bar(dataframe, dollar_per_bar = 0, number_bars = timeBarLen)
ohlcvDollar.head(10)

,open,high,low,close,volume,value_of_trades,price_mean,tick_count,price_mean_log_return
dates,,,,,,,,,
2020-01-02 09:30:00,130.6800,130.78,130.5000,130.67,24225,130.674262,130.631500,20,NaN
2020-01-02 09:30:05,130.6700,130.78,130.5200,130.52,4846,130.669480,130.660000,4,0.000218
2020-01-02 09:30:12,130.5200,130.74,130.5200,130.59,7182,130.604879,130.629840,35,-0.000231
2020-01-02 09:32:50,130.5700,130.59,130.5567,130.59,6121,130.575292,130.575780,15,-0.000414
2020-01-02 09:33:04,130.6100,130.61,130.5900,130.61,6100,130.599180,130.600769,13,0.000191
2020-01-02 09:33:19,130.6100,130.62,130.6100,130.62,300,130.613333,130.615000,2,0.000109
2020-01-02 09:33:22,130.6100,130.63,130.6100,130.63,48100,130.610042,130.620000,2,0.000038
2020-01-02 09:33:23,130.6300,130.67,130.5350,130.55,5832,130.602522,130.611625,20,-0.000064
2020-01-02 09:35:05,130.5500,130.62,130.4800,130.55,5465,130.545136,130.553600,23,-0.000444


In [1]:
countAverage = pd.DataFrame()
countAverage['Time'] = ohlcvTime.resample("1W")['tick_count'].mean()
countAverage['Tick'] = ohlcvTick.resample("1W")['tick_count'].mean()
countAverage['Volume'] = ohlcvVolume.resample("1W")['tick_count'].mean()
countAverage['Dollar'] = ohlcvDollar.resample("1W")['tick_count'].mean()
countAverage.head(10)

NameError: name 'pd' is not defined

In [9]:
countAverage

,Time,Tick,Volume,Dollar
dates,,,,
2020-01-05,13.676301,16.000000,13.917647,11.889447
2020-01-12,11.848214,16.000000,16.518672,14.067138
2020-01-19,12.035714,16.000000,19.536232,16.850000
2020-01-26,9.526786,16.000000,17.788889,15.247619
2020-02-02,15.601190,16.000000,20.552941,17.826531
2020-02-09,11.348214,16.000000,17.652778,15.627049
2020-02-16,7.669643,16.000000,18.539568,15.907407
2020-02-23,7.125000,16.000000,17.873134,15.653595
2020-03-01,45.872024,16.000000,23.860681,22.469388


In [23]:
# this should be deleted later, since it is supposed to be a part of the RiskLabAI utility functions
import plotly.graph_objects as go

def update_figure_layout(fig, title, xaxis_title, yaxis_title):
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        template="plotly_dark",
        plot_bgcolor='rgba(0,0,0,0)',  # Setting transparent background
        paper_bgcolor='rgba(0,0,0,0)', # Setting transparent background
        legend=dict(
            x=1,           # X position of the legend (1 is at the far right of the plot)
            y=1,           # Y position of the legend (1 is at the top of the plot)
            xanchor='auto',  # Anchor point of the legend
            yanchor='auto'   # Anchor point of the legend
        )
    )

In [22]:
# Assuming countAverage is a DataFrame you have already defined
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Time, name='Time'))
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Tick, name='Tick'))
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Volume, name='Volume'))
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Dollar, name='Dollar'))

# Use the function to update layout
update_figure_layout(
    fig,
    title="Standard Bars Frequency",
    xaxis_title="Date",
    yaxis_title="Frequency"
)

fig.write_json('Standard Bars Frequency.json')
fig.show()

In [13]:
# Select a smaller range
startDate = datetime.datetime(2020, 3, 1)
endDate = datetime.datetime(2020, 3, 5)
dataframe2 = dataframe[((dataframe.index >= startDate) & (dataframe.index <= endDate))][['price', 'size']]
dataframe2.drop_duplicates()
dataframe2['ret'] = np.log(dataframe2['price']) - np.log(dataframe2['price'].shift(1))
dataframe2['label'] = np.sign(dataframe2['ret'])
dataframe2['volumelabeled'] = dataframe2['label']*dataframe2['size']
dataframe2['dollarslabeled'] = dataframe2['volumelabeled']*dataframe2['price']
dataframe2.dropna(inplace = True)
dataframe2.head()

,price,size,ret,label,volumelabeled,dollarslabeled
dates,,,,,,
2020-03-02 09:00:12,114.7000,123,0.000087,1.0,123.0,1.410810e+04
2020-03-02 09:30:00,115.2800,26670,0.005044,1.0,26670.0,3.074518e+06
2020-03-02 09:30:07,115.3400,100,0.000520,1.0,100.0,1.153400e+04
2020-03-02 09:30:16,115.2200,100,-0.001041,-1.0,-100.0,-1.152200e+04
2020-03-02 09:30:26,115.2637,172,0.000379,1.0,172.0,1.982536e+04


In [14]:
ohlcv_time = generate_time_bar(dataframe2, frequency = "30Min")
ohlcv_tick = generate_tick_bar(dataframe2, ticks_per_bar = 1000)
ohlcv_volume = generate_volume_bar(dataframe2, volume_per_bar = 10000)
ohlcv_dollar = generate_dollar_bar(dataframe2, dollar_per_bar = 500000)

C:\Users\harian\AppData\Roaming\Python\Python311\site-packages\RiskLabAI\data\structures\data_structures_lopez.py:164: RuntimeWarning:

invalid value encountered in scalar divide



In [15]:
ohlcvinfo, thetasabsolute, thresholds = generate_information_driven_bars(dataframe2, bar_type = "dollar", tick_expected_initial = 500)

Completed: [------------------->] 100% - 0 minutes remaining.

In [17]:
ohlcvinfo.head()

,open,high,low,close,volume,value_of_trades,price_mean,tick_count,price_mean_log_return
dates,,,,,,,,,
2020-03-02 09:00:12,114.7000,115.2800,114.7000,115.2800,26793,115.277337,114.990000,2,NaN
2020-03-02 09:30:07,115.3400,115.3400,115.2200,115.2637,372,115.272463,115.274567,3,0.002472
2020-03-02 09:30:27,115.2530,115.2530,115.2530,115.2530,557,115.253000,115.253000,1,-0.000187
2020-03-02 09:31:00,115.5148,115.5148,115.5148,115.5148,238,115.514800,115.514800,1,0.002269
2020-03-02 09:31:02,115.5148,115.5148,115.4200,115.4541,588,115.463579,115.462967,3,-0.000449


In [19]:
thetasabsolute

array([  14108.1   , 3088625.7   ,   11534.    , ...,  564358.1716,
        564358.1716,  564358.1716])

In [20]:
thresholds 

array([      0.        , 1369245.11577499,   14108.1       , ...,
       4917515.44336638, 4917515.44336638, 4917515.44336638])